In [375]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [2]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [3]:
data=pd.read_csv('../processed_logs.csv',index_col=0)

In [407]:
data.shape

(575061, 157)

In [4]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,147,148,149,150,151,152,153,154,155,anomaly_label
0,3.000000,1.000000,3.000000,3.000000,3.000000,2.000000,1.000000,3.000000,3.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,3.000000,3.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,3.000000,3.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,3.000000,1.000000,3.000000,3.000000,3.000000,2.000000,1.000000,3.000000,3.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [5]:
X_train,X_test,y_train,y_test=model_selection.train_test_split(data.iloc[:,:-1],data.loc[:,'anomaly_label'],test_size=0.15,random_state=586)

In [6]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(488801, 156)
(488801,)
(86260, 156)
(86260,)


In [7]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,146,147,148,149,150,151,152,153,154,155
480256,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
108706,3.000000,1.000000,3.000000,3.000000,3.000000,2.000000,1.000000,3.000000,3.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
379236,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
573215,3.000000,1.000000,3.000000,3.000000,3.000000,2.000000,1.000000,3.000000,3.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
305262,3.000000,1.000000,3.000000,3.000000,3.000000,0.000000,0.000000,3.000000,3.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [8]:
print('Train percent anamolies:',str(np.round(y_train.mean(),4)))
print('Test percent anamolies:',str(np.round(y_test.mean(),4)))

Train percent anamolies: 0.0293
Test percent anamolies: 0.029


## Standardization

In [9]:
## Perform standardization

# Fit StandardScaler on training set
scaler=preprocessing.StandardScaler().fit(X_train)

# Apply standardization to train/test sets
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

# Examine mean/std of first 5
print('Train mean:',X_train_scaled.mean(axis=0)[0:5])
print('Test mean:',X_test_scaled.mean(axis=0)[0:5])

print('Train std:',X_train_scaled.std(axis=0)[0:5])
print('Test std:',X_test_scaled.std(axis=0)[0:5])

Train mean: [-7.26182494e-16  0.00000000e+00 -4.85647989e-16 -1.95442462e-16
 -2.94915334e-16]
Test mean: [0.00275988 0.         0.00505306 0.00404968 0.00250741]
Train std: [1. 0. 1. 1. 1.]
Test std: [1.00747758 0.         0.98275836 0.97977846 0.98379632]


## PCA (Dimensionality Reduction)

In [10]:
# Perform PCA on scaled data
pca=PCA()
pca.fit(X_train_scaled)

PCA()

In [11]:
# Examine cumulative proportion of variance explained
print(pca.explained_variance_ratio_.cumsum())

[0.04750859 0.08817697 0.11806787 0.14545955 0.17002872 0.19313692
 0.21487609 0.23661527 0.25835444 0.28009362 0.30183216 0.32262586
 0.34306334 0.36241976 0.38144819 0.40038449 0.41854496 0.43623694
 0.45290777 0.46849176 0.48363444 0.49826264 0.51281997 0.52717507
 0.54131139 0.55495502 0.56765768 0.57944152 0.59064633 0.60175483
 0.61155078 0.62073992 0.6298328  0.63868655 0.64684982 0.65481632
 0.66265446 0.67027904 0.67782159 0.68523934 0.69262409 0.69997499
 0.70725571 0.71451226 0.7217616  0.72900897 0.7362554  0.7435018
 0.7507482  0.7579946  0.76524099 0.77248739 0.77973378 0.78698017
 0.79422656 0.80147295 0.80871934 0.81596574 0.82321213 0.83045852
 0.83770491 0.8449513  0.85219769 0.85944409 0.86669048 0.87393687
 0.88118326 0.88842918 0.89567265 0.90289983 0.9099805  0.91703115
 0.92384759 0.93039872 0.93664212 0.94271838 0.94787254 0.95220404
 0.9561483  0.959922   0.96354521 0.9670143  0.97039779 0.97350761
 0.97656712 0.97951951 0.98236829 0.98513706 0.98777851 0.98998

In [12]:
num_components=np.argmax(pca.explained_variance_ratio_.cumsum()>=0.99)
print('# Components to explain 99% of variance:',num_components)

# Components to explain 99% of variance: 90


In [13]:
# Get component scores for train/test sets
X_train_scores=pca.transform(X_train_scaled)[:,:num_components]
X_test_scores=pca.transform(X_test_scaled)[:,:num_components]
print(X_train_scores.shape)
print(X_test_scores.shape)

(488801, 90)
(86260, 90)


## Define weighted binary cross entropy loss function

In [225]:
def weighted_bce(y_true,y_pred):
    sample_weights=(y_true.values==1).astype(float)
    sample_weights=sample_weights*(1/np.mean(y_true))
    sample_weights=np.where(sample_weights==0,1,sample_weights)
    return metrics.log_loss(y_true=y_true,y_pred=y_pred,sample_weight=sample_weights)

## Logistic regression on first 90 PCs

In [230]:
# Perform logistic regression on component scores

In [231]:
# Define dictionary of metrics for CV
eval_metrics={'weighted_bce':metrics.make_scorer(weighted_bce),'accuracy':metrics.make_scorer(metrics.accuracy_score),'recall':metrics.make_scorer(metrics.recall_score),'precision':metrics.make_scorer(metrics.precision_score)}

In [234]:
# Get 5-fold CV estimates for logistic regression model
logistic_regressor_cv=model_selection.cross_validate(LogisticRegression(max_iter=1000,n_jobs=-1),X=X_train_scores,y=y_train,scoring=eval_metrics,n_jobs=-1,verbose=1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   35.7s remaining:   53.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   46.1s finished


In [235]:
logistic_regressor_cv

{'fit_time': array([41.62140703, 44.70747209, 35.00737   , 34.34562516, 33.01395297]),
 'score_time': array([0.06046104, 0.04790878, 0.07120609, 0.09732294, 0.09866214]),
 'test_weighted_bce': array([0.05894362, 0.0914829 , 0.1376988 , 0.14309481, 0.12045256]),
 'test_accuracy': array([0.9996829 , 0.99918167, 0.99925327, 0.99928396, 0.99956015]),
 'test_recall': array([0.99686192, 0.99546564, 0.99267527, 0.99232647, 0.99337286]),
 'test_precision': array([0.99236376, 0.97706265, 0.98205659, 0.98340823, 0.99164345])}

In [236]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in logistic_regressor_cv.items()}

{'fit_time': 37.73917,
 'score_time': 0.07511,
 'test_weighted_bce': 0.11033,
 'test_accuracy': 0.99939,
 'test_recall': 0.99414,
 'test_precision': 0.98531}

## QDA on first 90 PCs

In [237]:
qda_model=QuadraticDiscriminantAnalysis()
# Get 5-fold CV estimates for QDA
qda_model_cv=model_selection.cross_validate(qda_model,X=X_train_scores,y=y_train,scoring=eval_metrics,n_jobs=-1,verbose=1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   20.1s remaining:   30.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   20.3s finished


In [238]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in qda_model_cv.items()}

{'fit_time': 18.95564,
 'score_time': 0.354,
 'test_weighted_bce': 0.90334,
 'test_accuracy': 0.99653,
 'test_recall': 0.95048,
 'test_precision': 0.93374}

## KNN on first 90 PCs

In [315]:
# Create KNN model with default params
knn_model=KNeighborsClassifier(n_jobs=-1)

In [317]:
# Get 5-fold CV estimates for default KNN model
# Use 10% of training set (computationally expensive)
knn_model_cv=model_selection.cross_validate(knn_model,X=X_train_scores[:int(len(X_train_scores)*0.1),:],y=y_train[:int(len(X_train_scores)*0.1)],scoring=eval_metrics,n_jobs=-1,verbose=1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   25.7s remaining:   38.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   25.8s finished


In [318]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in knn_model_cv.items()}

{'fit_time': 0.01714,
 'score_time': 24.27379,
 'test_weighted_bce': 0.12314,
 'test_accuracy': 0.99959,
 'test_recall': 0.99318,
 'test_precision': 0.99319}

In [324]:
# Tune k hyperparameter
k_vals=[]
knn_cv_weighted_bce=[]
counter=0
max_k=50
for i in np.arange(1,max_k+1):
    counter+=1
    k_vals.append(i)
    # Define KNN model
    knn_model_tuner=KNeighborsClassifier(n_neighbors=i,n_jobs=-1)
    # Get CV estimate of weighted binary cross entropy loss
    knn_model_tuner_cv=model_selection.cross_validate(knn_model_tuner,X=X_train_scores[:int(len(X_train_scores)*0.1),:],y=y_train[:int(len(X_train_scores)*0.1)],scoring=eval_metrics,n_jobs=-1)
    knn_cv_weighted_bce.append(np.mean(knn_model_tuner_cv['test_weighted_bce']))
    print('Performed iteration '+str(counter)+' of '+str(max_k))
knn_grid_results=pd.DataFrame({'k_val':k_vals,'cv_weighted_bce':knn_cv_weighted_bce})

Performed iteration 1 of 50
Performed iteration 2 of 50
Performed iteration 3 of 50
Performed iteration 4 of 50


/Users/ericphillips/miniforge3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Performed iteration 5 of 50
Performed iteration 6 of 50
Performed iteration 7 of 50
Performed iteration 8 of 50
Performed iteration 9 of 50
Performed iteration 10 of 50
Performed iteration 11 of 50
Performed iteration 12 of 50
Performed iteration 13 of 50
Performed iteration 14 of 50
Performed iteration 15 of 50
Performed iteration 16 of 50
Performed iteration 17 of 50
Performed iteration 18 of 50
Performed iteration 19 of 50
Performed iteration 20 of 50
Performed iteration 21 of 50
Performed iteration 22 of 50
Performed iteration 23 of 50
Performed iteration 24 of 50
Performed iteration 25 of 50
Performed iteration 26 of 50
Performed iteration 27 of 50
Performed iteration 28 of 50
Performed iteration 29 of 50
Performed iteration 30 of 50
Performed iteration 31 of 50
Performed iteration 32 of 50
Performed iteration 33 of 50
Performed iteration 34 of 50


/Users/ericphillips/miniforge3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Performed iteration 35 of 50
Performed iteration 36 of 50
Performed iteration 37 of 50
Performed iteration 38 of 50
Performed iteration 39 of 50
Performed iteration 40 of 50
Performed iteration 41 of 50
Performed iteration 42 of 50
Performed iteration 43 of 50
Performed iteration 44 of 50
Performed iteration 45 of 50
Performed iteration 46 of 50
Performed iteration 47 of 50
Performed iteration 48 of 50
Performed iteration 49 of 50
Performed iteration 50 of 50


In [328]:
knn_grid_results.sort_values('cv_weighted_bce').head(10)

,k_val,cv_weighted_bce
0,1,0.051057
2,3,0.075311
47,48,0.094361
46,47,0.094361
48,49,0.094720
49,50,0.094720
1,2,0.121745
4,5,0.123140
6,7,0.135067
5,6,0.135067


In [331]:
# Create KNN model with tuned k param
tuned_knn_model=KNeighborsClassifier(n_neighbors=3,n_jobs=-1)

In [332]:
# Get 5-fold CV estimates for default KNN model on whole dataset
tuned_knn_model_cv=model_selection.cross_validate(tuned_knn_model,X=X_train_scores,y=y_train,scoring=eval_metrics,n_jobs=-1,verbose=1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 34.6min remaining: 51.9min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 34.7min finished


In [334]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in tuned_knn_model_cv.items()}

{'fit_time': 0.18673,
 'score_time': 2072.15979,
 'test_weighted_bce': 0.02739,
 'test_accuracy': 0.99986,
 'test_recall': 0.99854,
 'test_precision': 0.99666}

## Random forest on first 90 PCs

In [239]:
# Create random forest with default params
random_forest=RandomForestClassifier(n_jobs=-1,random_state=586)

In [241]:
# Get 5-fold CV estimates for default random forest
random_forest_cv=model_selection.cross_validate(random_forest,X=X_train_scores,y=y_train,scoring=eval_metrics,n_jobs=-1,verbose=1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   59.2s remaining:  1.5min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   59.6s finished


In [242]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in random_forest_cv.items()}

{'fit_time': 58.08247,
 'score_time': 0.3241,
 'test_weighted_bce': 0.01887,
 'test_accuracy': 0.99987,
 'test_recall': 0.99902,
 'test_precision': 0.99659}

In [243]:
## Tune random forest hyperparameters using OOB weighted binary cross entropy loss
rf_params={'criterion':['gini','entropy'],
           'max_features':['auto','sqrt','log2'],
           'max_depth':[30,40,50,60,None]}

In [245]:
# Implement random grid search
counter=0
num_iter=15
rf_grid_results=[]
for i in range(num_iter):
    counter+=1
    # Select random parameter set
    param_choice={key:np.random.choice(val) for key,val in rf_params.items()}
    # Define model with param choices
    random_forest_tuner=RandomForestClassifier(
                           criterion=param_choice['criterion'],
                           max_features=param_choice['max_features'],
                           max_depth=param_choice['max_depth'],
                           oob_score=True,
                           n_jobs=-1,
                           verbose=1,
                           random_state=586)
    # Fit model
    random_forest_tuner.fit(X_train_scores,y_train)
    # Compute OOB binary weighted cross entropy loss
    param_choice['oob_weighted_bce']=weighted_bce(y_train,random_forest_tuner.oob_decision_function_[:, 1])
    rf_grid_results.append(param_choice)
    print('Performed iteration '+str(counter)+' of '+str(num_iter))
rf_grid_results=pd.DataFrame(rf_grid_results)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   14.2s finished


Performed iteration 1 of 15


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    9.2s finished


Performed iteration 2 of 15


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   13.3s finished


...

Performed iteration 13 of 15


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   14.2s finished


Performed iteration 14 of 15


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   14.8s finished


Performed iteration 15 of 15


In [248]:
rf_grid_results.sort_values('oob_weighted_bce')

,criterion,max_features,max_depth,oob_weighted_bce
2,entropy,auto,40.000000,0.003522
3,entropy,sqrt,30.000000,0.003522
6,entropy,auto,nan,0.003522
7,entropy,sqrt,30.000000,0.003522
12,entropy,sqrt,nan,0.003522
10,gini,log2,nan,0.003659
1,entropy,log2,30.000000,0.004700
5,entropy,log2,nan,0.004700
0,gini,sqrt,30.000000,0.004714
4,gini,sqrt,nan,0.004714


In [249]:
# Fit random forest using tuned hyperparameters
best_rf_params=rf_grid_results.sort_values('oob_weighted_bce').iloc[0,:]
tuned_random_forest=RandomForestClassifier(criterion=best_rf_params['criterion'],
                                           max_features=best_rf_params['max_features'],
                                           max_depth=best_rf_params['max_depth'],
                                           n_jobs=-1,
                                           random_state=586)

In [251]:
# Get 5-fold CV estimates for tuned random forest
tuned_random_forest_cv=model_selection.cross_validate(tuned_random_forest,X=X_train_scores,y=y_train,scoring=eval_metrics,n_jobs=-1,verbose=1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   50.0s remaining:  1.3min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   50.6s finished


In [252]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in tuned_random_forest_cv.items()}

{'fit_time': 48.84725,
 'score_time': 0.49096,
 'test_weighted_bce': 0.01769,
 'test_accuracy': 0.99987,
 'test_recall': 0.99909,
 'test_precision': 0.99652}

## Gradient boosting classifier on first 90 PCs

In [256]:
# Create gradient boosting classifier with default params
gradient_boosting=GradientBoostingClassifier(random_state=586)

In [257]:
# Get 5-fold CV estimates for gradient boosting classifier
gradient_boosting_cv=model_selection.cross_validate(gradient_boosting,X=X_train_scores,y=y_train,scoring=eval_metrics,n_jobs=-1,verbose=1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  5.3min remaining:  7.9min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  5.3min finished


In [258]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in gradient_boosting_cv.items()}

{'fit_time': 313.38383,
 'score_time': 0.28748,
 'test_weighted_bce': 0.01884,
 'test_accuracy': 0.99987,
 'test_recall': 0.99902,
 'test_precision': 0.99666}

In [261]:
## Tune gradient boosting classifier hyperparameters
gb_params={'n_estimators':[100,200,300,400,500],
           'subsample':[0.4,0.6,0.8,1],
           'max_depth':[4,6,8,10]}

In [275]:
# Implement random grid search
# Using 10% sample of dataset -- too computationally expensive otherwise
counter=0
num_iter=15
gb_grid_results=[]
for i in range(num_iter):
    counter+=1
    # Select random parameter set
    param_choice={key:np.random.choice(val) for key,val in gb_params.items()}
    # Define model with param choices
    gradient_boosting_tuner=GradientBoostingClassifier(
                           n_estimators=param_choice['n_estimators'],
                           subsample=param_choice['subsample'],
                           max_depth=param_choice['max_depth'],
                           learning_rate=10/param_choice['n_estimators'],
                           random_state=586)
    # Get CV estimate of weighted binary cross entropy loss
    gradient_boosting_tuner_cv=model_selection.cross_validate(gradient_boosting_tuner,X=X_train_scores[:int(len(X_train_scores)*0.1),:],y=y_train[:int(len(X_train_scores)*0.1)],scoring=eval_metrics,n_jobs=-1,verbose=1)
    param_choice['learning_rate']=10/param_choice['n_estimators']
    param_choice['cv_weighted_bce']=np.mean(gradient_boosting_tuner_cv['test_weighted_bce'])
    gb_grid_results.append(param_choice)
    print('Performed iteration '+str(counter)+' of '+str(num_iter))
gb_grid_results=pd.DataFrame(gb_grid_results)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.6min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Performed iteration 1 of 15


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.3min remaining:  1.9min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.3min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Performed iteration 2 of 15


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  2.2min remaining:  3.3min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   24.8s remaining:   37.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   25.0s finished


Performed iteration 14 of 15


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  2.1min remaining:  3.1min


Performed iteration 15 of 15


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.1min finished


In [280]:
gb_grid_results.sort_values('cv_weighted_bce')

,n_estimators,subsample,max_depth,learning_rate,cv_weighted_bce
6,100,0.800000,8,0.100000,0.074275
4,500,0.800000,4,0.020000,0.086561
0,400,1.000000,4,0.025000,0.086601
5,400,1.000000,4,0.025000,0.086601
8,200,0.400000,10,0.050000,0.086641
7,400,0.800000,10,0.025000,0.098169
13,100,1.000000,10,0.100000,0.098169
3,300,0.600000,8,0.033333,0.098209
11,500,0.400000,8,0.020000,0.098608
1,300,1.000000,4,0.033333,0.098887


In [281]:
best_gb_params=gb_grid_results.sort_values('cv_weighted_bce').iloc[0,:]

In [283]:
gb_lr_params={'learning_rate':[0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15]}

In [305]:
# Implement exhaustive grid search to further tune learning rate
counter=0
gb_lr_grid_results=[]
for i in range(len(gb_lr_params['learning_rate'])):
    counter+=1
    # Select parameter
    param_choice={key:val[i] for key,val in gb_lr_params.items()}
    # Define model with param choices
    gradient_boosting_lr_tuner=GradientBoostingClassifier(
                           n_estimators=int(best_gb_params['n_estimators']),
                           subsample=best_gb_params['subsample'],
                           max_depth=int(best_gb_params['max_depth']),
                           learning_rate=param_choice['learning_rate'],
                           random_state=586)
    # Get CV estimate of weighted binary cross entropy loss
    gradient_boosting_lr_tuner_cv=model_selection.cross_validate(gradient_boosting_lr_tuner,X=X_train_scores[:int(len(X_train_scores)*0.1),:],y=y_train[:int(len(X_train_scores)*0.1)],scoring=eval_metrics,n_jobs=-1,verbose=1)
    param_choice['cv_weighted_bce']=np.mean(gradient_boosting_lr_tuner_cv['test_weighted_bce'])
    gb_lr_grid_results.append(param_choice)
    print('Performed iteration '+str(counter)+' of '+str(len(gb_lr_params['learning_rate'])))
gb_lr_grid_results=pd.DataFrame(gb_lr_grid_results)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   23.1s remaining:   34.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   24.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Performed iteration 1 of 10


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   24.2s remaining:   36.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   26.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Performed iteration 2 of 10


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   25.3s remaining:   37.9s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   26.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  4.6min remaining:  6.9min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  4.6min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Performed iteration 9 of 10


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   30.5s remaining:   45.7s


Performed iteration 10 of 10


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   32.0s finished


In [307]:
gb_lr_grid_results.sort_values('cv_weighted_bce')

,learning_rate,cv_weighted_bce
5,0.100000,0.074275
8,0.140000,0.074275
7,0.130000,0.074633
9,0.150000,0.074992
1,0.060000,0.086242
3,0.080000,0.086242
4,0.090000,0.086601
0,0.050000,0.098528
2,0.070000,0.098528
6,0.120000,0.098568


In [310]:
# Create gradient boosting classifier with tuned params
tuned_gradient_boosting=GradientBoostingClassifier(
                                                  n_estimators=int(best_gb_params['n_estimators']),
                                                  subsample=best_gb_params['subsample'],
                                                  max_depth=int(best_gb_params['max_depth']),
                                                  learning_rate=gb_lr_grid_results.sort_values('cv_weighted_bce').iloc[0]['learning_rate'],
                                                  random_state=586)

In [311]:
# Get 5-fold CV estimates for tuned gradient boosting classifier
tuned_gradient_boosting_cv=model_selection.cross_validate(tuned_gradient_boosting,X=X_train_scores,y=y_train,scoring=eval_metrics,n_jobs=-1,verbose=1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  9.9min remaining: 14.9min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 10.1min finished


In [312]:
# Get avg error metrics from folds
{key:np.round(np.mean(val),5) for key,val in tuned_gradient_boosting_cv.items()}

{'fit_time': 598.12489,
 'score_time': 0.29889,
 'test_weighted_bce': 0.01628,
 'test_accuracy': 0.99988,
 'test_recall': 0.99916,
 'test_precision': 0.99687}

## Comparison of results

In [376]:
all_results=[]
for model in [logistic_regressor_cv,qda_model_cv,knn_model_cv,tuned_knn_model_cv,random_forest_cv,tuned_random_forest_cv,gradient_boosting_cv,tuned_gradient_boosting_cv]:
    all_results.append({key:np.round(np.mean(val),5) for key,val in model.items()})
all_results_df=pd.DataFrame(all_results)
all_results_df['test_f1_score']=(2*all_results_df['test_precision']*all_results_df['test_recall'])/(all_results_df['test_precision']+all_results_df['test_recall'])
all_results_df['model']=['Logistic Regression','QDA','KNN','Tuned KNN','Random Forest','Tuned Random Forest','Gradient Boosting','Tuned Gradient Boosting']
all_results_df=all_results_df.rename(columns={'fit_time':'Avg Fit Time','score_time':'Avg Score Time','test_weighted_bce':'CV Binary Cross Entropy Loss','test_accuracy':'CV Accuracy','test_recall':'CV Recall','test_precision':'CV Precision','test_f1_score':'CV F1 Score','model':'Model'})
all_results_df=all_results_df.sort_values('CV F1 Score',ascending=False)[['Model','CV F1 Score','CV Binary Cross Entropy Loss','CV Accuracy','CV Recall','CV Precision','Avg Fit Time','Avg Score Time']]
all_results_df

,Model,CV F1 Score,CV Binary Cross Entropy Loss,CV Accuracy,CV Recall,CV Precision,Avg Fit Time,Avg Score Time
7,Tuned Gradient Boosting,0.998014,0.016280,0.999880,0.999160,0.996870,598.124890,0.298890
6,Gradient Boosting,0.997839,0.018840,0.999870,0.999020,0.996660,313.383830,0.287480
4,Random Forest,0.997804,0.018870,0.999870,0.999020,0.996590,58.082470,0.324100
5,Tuned Random Forest,0.997803,0.017690,0.999870,0.999090,0.996520,48.847250,0.490960
3,Tuned KNN,0.997599,0.027390,0.999860,0.998540,0.996660,0.186730,2072.159790
2,KNN,0.993185,0.123140,0.999590,0.993180,0.993190,0.017140,24.273790
0,Logistic Regression,0.989705,0.110330,0.999390,0.994140,0.985310,37.739170,0.075110
1,QDA,0.942036,0.903340,0.996530,0.950480,0.933740,18.955640,0.354000


## Evaluate best model (tuned gradient boosting classifier) on test set

In [350]:
# Define gradient boosting model with tuned hyperparameters
tuned_gb=GradientBoostingClassifier(
                                   n_estimators=int(best_gb_params['n_estimators']),
                                   subsample=best_gb_params['subsample'],
                                   max_depth=int(best_gb_params['max_depth']),
                                   learning_rate=gb_lr_grid_results.sort_values('cv_weighted_bce').iloc[0]['learning_rate'],
                                   random_state=586)

In [351]:
# Fit model
tuned_gb.fit(X_train_scores,y_train)

GradientBoostingClassifier(max_depth=8, random_state=586, subsample=0.8)

In [411]:
# Generate predictions+probabilities on test set
test_preds=tuned_gb.predict(X_test_scores)
test_probs=tuned_gb.predict_proba(X_test_scores)

# Compute error metrics
test_bce=weighted_bce(y_test,test_probs)
test_acc=metrics.accuracy_score(y_test,test_preds)
test_recall=metrics.recall_score(y_test,test_preds)
test_precision=metrics.precision_score(y_test,test_preds)

# Compile results into DataFrame
test_results=pd.DataFrame({'Weighted Binary Cross Entropy':test_bce,'Accuracy':test_acc,'Recall':test_recall,'Precision':test_precision},index=[0])
test_results['F1 Score']=(2*test_results['Precision']*test_results['Recall'])/(test_results['Precision']+test_results['Recall'])
test_results

,Weighted Binary Cross Entropy,Accuracy,Recall,Precision,F1 Score
0,0.004872,0.999872,0.998401,0.997206,0.997803
